In [1]:
import os
import requests
from tools import quadratic_calculator, fib_list, convert_currency
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate


load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
#initialize LLM 
llm = ChatOpenAI(
    model="gpt-5-nano"
)

In [3]:
#bind llm to tools
llm_with_tools = llm.bind_tools([quadratic_calculator, fib_list, convert_currency])


In [33]:
#generate the prompt 
prompt = ChatPromptTemplate([
    ("system", "You are a helpful agent that will be given a set of tools. Your only job is to execute the right one"),
    ("user", "{query}")
])

# query
#query = "can you help me find the solution for x**2 + 8x + 16?"
#query = "what are the first 20 numbers in the fibonacci sequence?"
query = "how much is 10000000000 us dollars in us dollars?"


In [37]:
#retrieve answer from tool caller
res = llm_with_tools.invoke(prompt.format_messages(query=query)).tool_calls

print(res)

[{'name': 'convert_currency', 'args': {'amount': 10000000000, 'fromCurrency': 'USD', 'toCurrency': 'USD'}, 'id': 'call_tbrffTaMPdwvRcJxpT2kVhRc', 'type': 'tool_call'}]


In [38]:
#make sure tool was called
if not res:
    raise ValueError("No tool called")


#figure out which tool was called
fn = FUNCTION_NAMES[res[0]["name"]]

#call the tool 
ans = fn.invoke(res[0]["args"])

10000000000.0


In [39]:
print(ans)

10000000000.0


In [40]:
FUNCTION_NAMES = {
    "quadratic_calculator": quadratic_calculator,
    "fib_list": fib_list,
    "convert_currency": convert_currency
}